In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from data_utils import get_data
from vae import VAE
import os
import numpy as np
import torch

In [ ]:
# Extraire les images et les labels
images, labels = [], []

events = ['cards', 
          #'center'
          #'free-kick'
         ]

train_images, train_labels = get_data(folder='train', events=events)

In [ ]:
vae = VAE()

print('input shape')
print(train_images[:2].shape)
vae(train_images[:2])